<a href="https://colab.research.google.com/github/Vidatsa/Stochastic-Nature-of-Molecular-Motors/blob/main/SSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba



# Plotting modules
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()


In [185]:
#Initial conditions
x1=1
x2=x3=x4=x5=x6=x7=x8=x9=0
#rates
one = 870,   
two = 1000,                     
three = 13.1,                        
four = 1000,
five = 870,
six = 1000,
seven = 13.1,
eight = 1000,
nine = 100,
ten = 100,
eleven = 100


In [186]:
# Column 0 is change in m, column 1 is change in p
simple_update = np.array([[-1, 1, 0, 0, 0, 0, 0, 0, 0],   # Make mRNA transcript
                          [0, -1, 1, 0, 0, 0, 0, 0, 0],  # Degrade mRNA
                          [0, 0, -1, 1, 0, 0, 0, 0, 0],   # Make protein
                          [0, 0, 0, -1, 1, 0, 0, 0, 0],
                          [0, 0, 0, 0, -1, 1, 0, 0, 0],
                          [0, 0, 0, 0, 0, -1, 1, 0, 0],
                          [0, 0, 0, 0, 0, 0, -1, 1, 0],
                          [0, 0, 0, 0, 0, 0, 0, -1, 1],
                          [1, 0, 0, 0, 0, 0, 0, 0, -1],
                          [0, 0, -1, 0, 0, 0, 0, 0, 1],
                          [0, 0, 0, -1, 0, 0, 0, 0, 1]], # Degrade protein
                         dtype=np.int)

<ipython-input-186-31ec09632a62>:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [187]:
def simple_propensity(propensities, population, t, one, two, three, four, five, six, seven, eight, nine, ten, eleven):
    """Updates an array of propensities given a set of parameters
    and an array of populations.
    """
    # Unpack population
    x1, x2, x3, x4, x5, x6, x7, x8, x9 = population
    propensities=np.ones(11)
    # Update propensities
        # Make mRNA transcript
    propensities[0] = one         # Degrade mRNA
    propensities[1] =  two*x1  # Make protein
    propensities[2] =  three*x2  # Degrade protein
    propensities[3] = four*x3
    propensities[4] = five*x4
    propensities[5] =six*x5
    propensities[6] =  seven*x6
    propensities[7] = eight*x7
    propensities[8] = nine*x9
    propensities[9] = ten*x3
    propensities[10] = eleven*x4
    return propensities





In [188]:
def sample_discrete_scipy(probs):
    """Randomly sample an index with probability given by probs."""
    return st.rv_discrete(values=(range(len(probs)), probs)).rvs()
  

In [189]:
def sample_discrete(probs):
    """Randomly sample an index with probability given by probs."""
    # Generate random number
    q = np.random.rand()
    
    # Find index
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1

In [190]:
# Make dummy probs
probs = np.array([0.1, 0.3, 0.4, 0.05, 0.15])

print('Result from scipy.stats:')
%timeit sample_discrete_scipy(probs)

print('\nResult from hand-coded method:')
%timeit sample_discrete(probs)

Result from scipy.stats:
758 µs ± 6.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Result from hand-coded method:
1.44 µs ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [191]:
def gillespie_draw(propensity_func, propensities, population, t, args=()):
    """
    Draws a reaction and the time it took to do that reaction.
    
    Parameters
    ----------
    propensity_func : function
        Function with call signature propensity_func(population, t, *args)
        used for computing propensities. This function must return
        an array of propensities.
    population : ndarray
        Current population of particles
    t : float
        Value of the current time.
    args : tuple, default ()
        Arguments to be passed to `propensity_func`.
        
    Returns
    -------
    rxn : int
        Index of reaction that occured.
    time : float
        Time it took for the reaction to occur.
    """
    # Compute propensities
    propensity_func(propensities, population, t, *args)
    
    # Sum of propensities
    props_sum = propensities.sum()
    
    # Compute next time
    time = np.random.exponential(1.0 / props_sum)
    
    # Compute discrete probabilities of each reaction
    rxn_probs = propensities / props_sum
    
    # Draw reaction from this distribution
    rxn = sample_discrete(rxn_probs)
    
    return rxn, time



In [192]:
def gillespie_ssa(propensity_func, update, population_0, time_points,args):
    """
    Uses the Gillespie stochastic simulation algorithm to sample
    from probability distribution of particle counts over time.
    
    Parameters
    ----------
    propensity_func : function
        Function of the form f(params, t, population) that takes the current
        population of particle counts and return an array of propensities
        for each reaction.
    update : ndarray, shape (num_reactions, num_chemical_species)
        Entry i, j gives the change in particle counts of species j
        for chemical reaction i.
    population_0 : array_like, shape (num_chemical_species)
        Array of initial populations of all chemical species.
    time_points : array_like, shape (num_time_points,)
        Array of points in time for which to sample the probability
        distribution.
    args : tuple, default ()
        The set of parameters to be passed to propensity_func.        

    Returns
    -------
    sample : ndarray, shape (num_time_points, num_chemical_species)
        Entry i, j is the count of chemical species j at time
        time_points[i].
    """

    # Initialize output
    pop_out = np.empty((len(time_points), update.shape[1]), dtype=np.int)#101,9

    # Initialize and perform simulation
    i_time = 1
    i = 0
    t = time_points[0]
    population = population_0.copy()
    pop_out[0,:] = population
    propensities = np.zeros(update.shape[0]) #11
    while i < len(time_points):
        while t < time_points[i_time]:
            # draw the event and time step
            event, dt = gillespie_draw(propensity_func, propensities, population, t, args)
                
            # Update the population
            population_previous = population.copy()
            population += update[event,:]
                
            # Increment time
            t += dt

        # Update the index
        i = np.searchsorted(time_points > t, True)
        
        # Update the population
        pop_out[i_time:min(i,len(time_points))] = population_previous
        
        # Increment index
        i_time = i
                           
    return pop_out

In [193]:
# Specify parameters for calculation
args = (870, 1000, 13.1,1000,870,1000,13.1,1000,100,100,100)
time_points = np.linspace(0, 50, 101)
population_0 = np.zeros(9, dtype=int)
size = 101

# Seed random number generator for reproducibility
np.random.seed(42)

# Initialize output array
samples = np.empty((size, len(time_points), 9), dtype=int)

# Run the calculations
for i in tqdm.tqdm_notebook(range(size)):
    samples[i,:,:] = gillespie_ssa(simple_propensity, simple_update,
                                population_0, time_points, args=args)

<ipython-input-193-fb8226d2e5d9>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(range(size)):


  0%|          | 0/101 [00:00<?, ?it/s]

<ipython-input-192-d535eb14e640>:31: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pop_out = np.empty((len(time_points), update.shape[1]), dtype=np.int)#101,9
<ipython-input-191-963a51cf8cb7>:32: RuntimeWarning: divide by zero encountered in double_scalars
  time = np.random.exponential(1.0 / props_sum)
<ipython-input-191-963a51cf8cb7>:35: RuntimeWarning: invalid value encountered in true_divide
  rxn_probs = propensities / props_sum


In [194]:
# Set up plots
plots = [bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x1'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x2'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x3'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x4'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x5'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x6'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x7'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x8'),
         bokeh.plotting.figure(plot_width=300,
                               plot_height=200,
                               x_axis_label='dimensionless time',
                               y_axis_label='x9')]

# Plot trajectories and mean
for i in [0, 8]:
    for x in samples[:,:,i]:
        plots[i].line(time_points, x, line_width=0.3, 
                      alpha=0.2, line_join='bevel')
    plots[i].line(time_points, samples[:,:,i].mean(axis=0),
                  line_width=6, color='orange', line_join='bevel')

# Link axes
plots[0].x_range = plots[1].x_range =  plots[2].x_range = plots[3].x_range = plots[4].x_range = plots[5].x_range = plots[6].x_range = plots[7].x_range = plots[8].x_range
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=9))
print(samples[:,:,:])

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
